In [55]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import stats
import seaborn as sns

In [56]:
df=pd.read_csv(r'D:\文件\学习\数据集\HR\data\HR.csv')
#删除缺失值
df=df.dropna()
#删除department中的错误类别sale
df=df[df['department']!='sale']

In [57]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_monthly_hours  14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.3+ MB


In [59]:
#离散值数值化
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
department_file=label_encoder.fit_transform(df['department'].values.reshape(-1,1)).reshape(1,-1)[0]

In [60]:
department_mapping={index:label for index,label in enumerate(label_encoder.classes_)}
department_mapping

{0: 'IT',
 1: 'RandD',
 2: 'accounting',
 3: 'hr',
 4: 'management',
 5: 'marketing',
 6: 'product_mng',
 7: 'sales',
 8: 'support',
 9: 'technical'}

In [61]:
#哑变量
department_dummy=pd.get_dummies(df['department'],prefix='department')
department_dummy

,department_IT,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical
0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
14994,0,0,0,0,0,0,0,0,1,0
14995,0,0,0,0,0,0,0,0,1,0
14996,0,0,0,0,0,0,0,0,1,0
14997,0,0,0,0,0,0,0,0,1,0


In [62]:
#二值化
# satisfaction_level
from sklearn.preprocessing import Binarizer
sat_bin=Binarizer(threshold=0.5)
sat_bin_code=sat_bin.fit_transform(df['satisfaction_level'].values.reshape(-1,1)).reshape(1,-1)[0]

In [63]:
sat_bin_code

array([0., 1., 0., ..., 0., 0., 0.])

In [64]:
#多项式增加维度
from sklearn.preprocessing import PolynomialFeatures
df_col=df.iloc[:,2:4]
num_and_hour=PolynomialFeatures(degree=3,interaction_only=False,include_bias=False)
res=num_and_hour.fit_transform(df_col)
res=pd.DataFrame(res)
res

,0,1,2,3,4,5,6,7,8
0,2.0,157.0,4.0,314.0,24649.0,8.0,628.0,49298.0,3869893.0
1,5.0,262.0,25.0,1310.0,68644.0,125.0,6550.0,343220.0,17984728.0
2,7.0,272.0,49.0,1904.0,73984.0,343.0,13328.0,517888.0,20123648.0
3,5.0,223.0,25.0,1115.0,49729.0,125.0,5575.0,248645.0,11089567.0
4,2.0,159.0,4.0,318.0,25281.0,8.0,636.0,50562.0,4019679.0
...,...,...,...,...,...,...,...,...,...
14994,2.0,151.0,4.0,302.0,22801.0,8.0,604.0,45602.0,3442951.0
14995,2.0,160.0,4.0,320.0,25600.0,8.0,640.0,51200.0,4096000.0
14996,2.0,143.0,4.0,286.0,20449.0,8.0,572.0,40898.0,2924207.0
14997,6.0,280.0,36.0,1680.0,78400.0,216.0,10080.0,470400.0,21952000.0


In [65]:
#连续值离散化
avg_hours=df['average_monthly_hours'].copy()

In [66]:
k=5
avg_hour_01=pd.cut(x=avg_hours,bins=k,labels=range(1,k+1))
avg_hour_01

0        2
1        4
2        5
3        3
4        2
        ..
14994    2
14995    2
14996    2
14997    5
14998    2
Name: average_monthly_hours, Length: 14999, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [67]:
#日期格式处理
train_data=pd.read_csv(r"D:\文件\学习\数据集\二手车交易\trainDataAfterCleaning.csv",index_col=False)

In [68]:
train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,price
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762,1850
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522,3600
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963,6222
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699,2400
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482,5200


In [69]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149999 entries, 0 to 149998
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             149999 non-null  int64  
 1   name               149999 non-null  int64  
 2   regDate            149999 non-null  int64  
 3   model              149999 non-null  float64
 4   brand              149999 non-null  int64  
 5   bodyType           149999 non-null  float64
 6   fuelType           149999 non-null  float64
 7   gearbox            149999 non-null  float64
 8   power              149999 non-null  int64  
 9   kilometer          149999 non-null  float64
 10  notRepairedDamage  149999 non-null  float64
 11  regionCode         149999 non-null  int64  
 12  seller             149999 non-null  int64  
 13  offerType          149999 non-null  int64  
 14  creatDate          149999 non-null  int64  
 15  v_0                149999 non-null  float64
 16  v_

In [77]:
year_data=[str(x)[:4] for x in train_data['regDate']]
month_data=[str(x)[4:6] for x in train_data['regDate']]
day_data=[str(x)[6:] for x in train_data['regDate']]
year_data=pd.DataFrame(year_data)
month_data=pd.DataFrame(month_data)
day_data=pd.DataFrame(day_data)
year_data.columns=['year']
month_data.columns=['month']
day_data.columns=['day']
train_data=pd.concat([train_data,year_data,month_data,day_data],axis=1)

In [80]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149999 entries, 0 to 149998
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             149999 non-null  int64  
 1   name               149999 non-null  int64  
 2   regDate            149999 non-null  int64  
 3   model              149999 non-null  float64
 4   brand              149999 non-null  int64  
 5   bodyType           149999 non-null  float64
 6   fuelType           149999 non-null  float64
 7   gearbox            149999 non-null  float64
 8   power              149999 non-null  int64  
 9   kilometer          149999 non-null  float64
 10  notRepairedDamage  149999 non-null  float64
 11  regionCode         149999 non-null  int64  
 12  seller             149999 non-null  int64  
 13  offerType          149999 non-null  int64  
 14  creatDate          149999 non-null  int64  
 15  v_0                149999 non-null  float64
 16  v_

In [79]:
train_data=train_data.astype({'year':'int64','month':'int64','day':'int64'})

In [83]:
train_data['month']=train_data['month'].replace(0,3)

In [96]:
train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_12,v_13,v_14,price,year,month,day,datetime,week,day_of_year
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,-2.420821,0.795292,0.914762,1850,2004,4,2,2004-04-02,14,93
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,-1.030483,-1.722674,0.245522,3600,2003,3,1,2003-03-01,9,60
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,1.565330,-0.832687,-0.229963,6222,2004,4,3,2004-04-03,14,94
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,-0.501868,-2.438353,-0.478699,2400,1996,9,8,1996-09-08,36,252
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.931110,2.834518,1.923482,5200,2012,1,3,2012-01-03,1,3


In [87]:
from datetime import datetime
date_list=[datetime(x,y,z) for x,y,z in zip(train_data.year,train_data.month,train_data.day)]
date_list=pd.DataFrame(date_list)
train_data['datetime']=date_list


In [95]:
train_data['year']=train_data['datetime'].apply(lambda x:x.year)
train_data['week']=train_data['datetime'].apply(lambda x:x.week)
train_data['day_of_year']=train_data['datetime'].apply(lambda x:x.dayofyear)